<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_2_tuning_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    700      0 --:--:-- --:--:-- --:--:--   698
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-2.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-2.validation.csv",header=None)
valid.head()

,0,1
0,2,left heel pain
1,4,Discoloration of skin [blotchy] on the neck
2,7,goiter
3,8,arthritis
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-2.test.csv",header=None)
test.head()

,0,1
0,2,right heel pain
1,4,black and blue marks
2,8,arthritice
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_pubmed_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.465225,4.799564,0.232616,10:50


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.669700,4.499007,0.261568,10:46


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.377179,4.462764,0.264719,10:52


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.321233,4.436665,0.266846,11:00


In [0]:
learn.save_encoder('pubmed_fold2_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_fold2_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.842715,3.421234,0.439314,01:11


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.980022,3.010109,0.505277,01:25


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.476771,2.802617,0.539578,03:22


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.341281,2.574256,0.571240,03:27
1,2.103310,2.474514,0.584433,03:16


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.908646,2.396358,0.585752,03:35
1,1.772942,2.287436,0.610818,03:24


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.730377,2.249756,0.624011,03:29
1,1.617995,2.180171,0.627968,03:23


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.575797,2.142144,0.624011,03:34
1,1.470178,2.104587,0.637203,03:22


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.417138,2.088886,0.646438,03:25
1,1.322407,2.062254,0.643799,03:31


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.309904,2.055970,0.645119,03:40
1,1.346070,2.050156,0.646438,03:15
2,1.253321,1.991204,0.650396,03:23
3,1.183641,1.979216,0.654354,03:22


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.188991,2.010731,0.660950,03:15
1,1.253560,2.006048,0.656992,03:20
2,1.161385,2.076000,0.668865,03:11
3,1.117929,2.042952,0.666227,03:31


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.081180,2.053019,0.655673,03:25
1,1.133903,2.059689,0.654354,03:12
2,1.068950,2.035851,0.666227,03:10
3,1.012059,1.998710,0.664908,03:25


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.044149,2.148188,0.655673,03:16
1,1.154041,2.094176,0.660950,03:13
2,0.998075,2.053455,0.662269,03:16
3,0.970372,2.076063,0.664908,03:25


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.053741,2.129155,0.667546,03:33
1,1.122950,2.165904,0.655673,03:12
2,0.974464,2.138427,0.660950,03:31
3,0.978094,2.150705,0.663588,03:37


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.998736,2.153007,0.659631,03:35
1,1.021549,2.132960,0.660950,03:21
2,1.033562,2.110008,0.662269,03:24
3,0.928772,2.065801,0.663588,03:22


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.949994,2.148793,0.658311,03:31
1,1.029485,2.077940,0.664908,03:10
2,1.033068,2.097457,0.672823,03:16
3,0.972367,2.145700,0.675462,03:26


In [0]:
# save the best model

learn.save_encoder('ask-2-tuning-pubmed')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [43]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 842, 418, 8, 8, 8, 418, 8, 318, 359, 659, 27, 30, 755, 584, 584, 584, 418, 384, 686, 37, 861, 37, 37, 37, 37, 189, 289, 289, 289, 289, 623, 462, 511, 49, 418, 49, 328, 473, 100, 100, 100, 100, 100, 100, 900, 157, 53, 418, 418, 53, 517, 58, 60, 418, 418, 418, 836, 876, 189, 289, 71, 71, 71, 470, 76, 289, 309, 309, 418, 418, 318, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 281, 281, 418, 182, 182, 182, 281, 788, 80, 80, 80, 80, 80, 80, 847, 418, 137, 418, 92, 92, 850, 94, 94, 94, 847, 847, 94, 94, 94, 94, 94, 418, 418, 645, 196, 100, 100, 100, 100, 108, 108, 108, 59, 275, 111, 111, 111, 111, 211, 111, 471, 120, 630, 418, 630, 630, 630, 630, 125, 125, 125, 125, 125, 125, 125, 125, 125, 778, 289, 131, 132, 418, 418, 418, 137, 139, 142, 143, 418, 418, 189, 152, 155, 157, 388, 162, 418, 166, 166, 289, 172, 172, 864, 864, 210, 524, 492, 196, 281, 182, 289, 289, 183, 183, 402, 187, 187, 187, 187, 187, 189, 462, 210, 

In [45]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

445
0.5138568129330254
